In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [3]:
csv_path   = '/omics/groups/OE0606/internal/jonas/genomic_data/snv_variant_calls'
file_names = '/omics/groups/OE0606/internal/jonas/genomic_data/shared_files.txt'
context_size= 255

## mannually adding small context columns

In [4]:
def add_smaller_contexts(df):
    # Ensure the context columns are stripped of whitespace
    df['seq_context_3'] = df['seq_context_3'].str.strip()
    df['seq_context_255'] = df['seq_context_255'].str.strip()

    # Iterate over each smaller size
    #print(df['seq_context_3'].head())
    for smaller_size in [5, 7, 9, 15, 31, 63, 127]:
        # Create a new column for each smaller context
        df[f'context_{smaller_size}'] = ""

        # Iterate through each row
        for idx, row in df.iterrows():
            context255 = row['seq_context_255']
            start_index = (len(context255) - smaller_size) // 2
            end_index = start_index + smaller_size

            # Extract the smaller context
            small_context = context255[start_index:end_index]
            df.at[idx, f'seq_context_{smaller_size}'] = small_context
        #print(df[f'context_{smaller_size}'].head())
    return df


In [ ]:
# Read the list of filenames from shared_files.txt
with open(file_names, 'r') as file:
    files = file.read().splitlines()

files = sorted(files, key=lambda f: os.path.getsize(os.path.join(csv_path, f + ".csv")))

i=0
for file in files:
    snvs=None
    try:
        snvs = pd.read_csv(os.path.join(csv_path, file + ".csv"), dtype={0: str})
        if 'seq_context_255' not in snvs.columns or 'seq_context_3' not in snvs.columns : 
            print(':(', end='  ')   
        else:
            if 'seq_context_127' in snvs.columns or 'context_127' in snvs.columns: 
                snvs.rename(columns={col: col.replace('context', 'seq_context') for col in snvs.columns if col.startswith('context')}, inplace=True)
                snvs.to_csv(os.path.join(csv_path, file + ".csv"), index=False)
                print(i, 'already', end=", ")
            else:
                extended_df = add_smaller_contexts(snvs)
                #print(extended_df.head(3))
                extended_df.to_csv(os.path.join(csv_path, file + ".csv"), index=False)
                print(i, 'finished', end=", ")
    except Exception as e: print(':(', end='  ')   

    i+=1


##  train/test split

### functions

In [5]:
def load_data(context_size):
    donors = []
    
    # Read the list of filenames from shared_files.txt
    with open(file_names, 'r') as file:
        files = file.read().splitlines()
    i=0
    for file in files:
        try:
            #if i<500:
            snvs = pd.read_csv(os.path.join(csv_path, file + ".csv"), dtype={0: str})[['REF', f'seq_context_{context_size}','ALT']]
            donors.append(snvs)
            print(i, end=' ')
            i+=1
        except Exception as e:
            print('.', end='')    
            #print(f"{e}", end='')    
    '''
    for i, snvs in enumerate(donors):
        print(f"DataFrame {i}:")
        print(snvs.head(3))
    '''
    return donors

In [9]:
def load_data_of_same_batch(context_size):
    donors = []
    
    # Read the list of filenames from shared_files.txt
    with open(file_names, 'r') as file:
        files = file.read().splitlines()
    i=0
    for file in files:
        try:
            df = pd.read_csv(os.path.join(csv_path, file + ".csv"), dtype={0: str})
            if 'seq_context_5' in df.columns:
                snvs =df[['REF', f'seq_context_{context_size}','ALT']]  
                donors.append(snvs)
                print(i, end=' ')
                i+=1
            else: print('x',end='')
        except Exception as e:
            print('.', end='')    
    return donors

In [6]:
def data_split(donors):
    train_dfs, val__hyperval_test_dfs = train_test_split(donors,                 test_size=0.2, random_state=42) 
    val_dfs, hyperval_test_dfs        = train_test_split(val__hyperval_test_dfs, test_size=0.5, random_state=42) 
    hyper_val_dfs, test_dfs           = train_test_split(     hyperval_test_dfs, test_size=0.5, random_state=42) 
    
    print(f'\n\n Split into {len(train_dfs)} dfs for training, {len(val_dfs)} dfs for validation (while training), \
         {len(hyper_val_dfs)} dfs for hyperparameter testing and {len(test_dfs)} for final testing  \n\n')
    
    return train_dfs, val_dfs, hyper_val_dfs, test_dfs

In [7]:
def input_data(df_list, split, context_size):
    #file_name = f"genomic_{split}_{context_size}_batch500.txt"
    file_name = f"genomic_{split}_{context_size}.txt"
    with open(file_name, 'w') as file:
        print('\n', split,':', len(df_list), "dataframes: ")
        i=0
        for df in df_list:
            print(i, end=' ')
            i=i+1
            for index, row in df[[f'seq_context_{context_size}', 'ALT']].iterrows():
                # Write the values to the file
                file.write(f"{row[f'seq_context_{context_size}']} {row['ALT']}\n")

In [11]:
def combined_data_generation(context_size):
    donors= load_data(context_size)
    train_dfs, val_dfs, hyper_val_dfs, test_dfs = data_split(donors)
    
    input_data(test_dfs,      'test',      context_size)
    input_data(hyper_val_dfs, 'hyper_val', context_size)
    input_data(val_dfs,       'val',       context_size)
    input_data(train_dfs,     'train',     context_size)

### workspace

#### context_size=5

In [31]:
combined_data_generation(5)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

#### context_size>=7

In [32]:
combined_data_generation(7)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

In [33]:
combined_data_generation(9)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

In [34]:
combined_data_generation(15)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

In [ ]:
combined_data_generation(31)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

In [7]:
combined_data_generation(63)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

In [8]:
combined_data_generation(127)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

##### 3

In [15]:
combined_data_generation(3)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

##### 255

In [16]:
combined_data_generation(255)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 .231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

# Manually adding 255, 3 and 1 base txt files based on 5 base files (quick fix)

In [11]:
def process_nucleotide_file(path_5base, path3_base, path1_base):
    os.makedirs(os.path.dirname(path3_base), exist_ok=True)
    os.makedirs(os.path.dirname(path1_base), exist_ok=True)
    
    with open(path_5base, 'r') as infile, \
         open(path3_base, 'w') as base_3, \
         open(path1_base, 'w') as base_1:
        
        for line in infile:
            sequence, nucleotide = line.strip().split()
            # Extract the middle 3 nucleotides
            middle_3 = sequence[1:4]
            # Extract the middle nucleotide
            middle_1 = sequence[2]
            
            # Write to respective files
            base_3.write(f"{middle_3} {nucleotide}\n")
            base_1.write(f"{middle_1} {nucleotide}\n")

In [12]:
for split in ('test','val','hyper_val','train'):
    path1_base=f'../nanoGPT/data/genomic_char/architecture_nr_1_context_size_1/genomic_{split}_1.txt'
    path3_base=f'../nanoGPT/data/genomic_char/architecture_nr_1_context_size_3/genomic_{split}_3.txt'
    path5_base=f'../nanoGPT/data/genomic_char/architecture_nr_1_context_size_5/genomic_{split}_5.txt'
    process_nucleotide_file(path5_base, path3_base, path1_base)

In [10]:
donors= load_data_of_same_batch(255)
train_dfs, val_dfs, hyper_val_dfs, test_dfs = data_split(donors)

input_data(test_dfs,      'test',      context_size)
input_data(hyper_val_dfs, 'hyper_val', context_size)
input_data(val_dfs,       'val',       context_size)
input_data(train_dfs,     'train',     context_size)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 x231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2